In [81]:
puzzle_input = open('input05_2.txt').read()

def parse_puzzle(puzzle_input):
    maps = []
    for map_i, map_token in enumerate(puzzle_input.strip().split('\n\n')):
        if map_i == 0: # seeds
            seeds = [int(seed) for seed in map_token.strip().split(': ')[1].split(' ')]
        else:
            ranges = []
            for range_token in map_token.strip().split('\n')[1:]:
                ranges.append(tuple(int(x) for x in range_token.strip().split(' ')))
            maps.append(ranges)
    return seeds, maps

seeds, maps = parse_puzzle(puzzle_input)

def apply_maps(x, maps):
    for map_ in maps:
        x = apply_map(x, map_)
    return x

def apply_map(x, map_):
    for dest, src, length in map_:
        if src <= x < src + length:
            return x - src + dest
    return x

# part 1
min(apply_maps(seed, maps) for seed in seeds)

389056265

In [94]:
seed_ranges = [(seeds[i], seeds[i + 1]) for i in range(0, len(seeds), 2)]

def partition_ranges(start1, length1, start2, length2):
    end1 = start1 + length1
    end2 = start2 + length2

    if start1 < end2 and end1 > start2:
        nonoverlapping_subranges = []
        overlapping_subranges = []

        if start1 < start2:
            nonoverlapping_subranges.append((start1, start2 - start1))
            
        if end1 > end2:
            nonoverlapping_subranges.append((end2, end1 - end2))
        
        if start2 <= start1 < end2:
            overlapping_start = start1
        else:
            overlapping_start = start2

        if start2 < end1 <= end2:
            overlapping_end = end1
        else:
            overlapping_end = end2

        return nonoverlapping_subranges, [(overlapping_start, overlapping_end - overlapping_start)]
    else:
        return [(start1, length1)], []

def range_apply_submap(start, length1, dest, src, length2):
    # assert range in submap bounds
    return (start - src + dest, length1)
    
def range_apply_map(unmapped_ranges, map_):
    mapped_ranges = []
    for dest, src, length in map_:
        unmapped_subranges_accum = []
        mapped_subranges_accum = []

        for unmapped_range in unmapped_ranges:
            nonoverlapping_subranges, overlapping_subranges = partition_ranges(unmapped_range[0], unmapped_range[1], src, length)

            unmapped_subranges_accum += nonoverlapping_subranges
            mapped_subranges_accum += [range_apply_submap(subrange[0], subrange[1], dest, src, length)
                                       for subrange in overlapping_subranges]
        # rebuild subranges
        unmapped_ranges = unmapped_subranges_accum
        mapped_ranges += mapped_subranges_accum
    return unmapped_ranges + mapped_ranges

def range_apply_maps(ranges, maps):
    for map_ in maps:
        ranges = range_apply_map(ranges, map_)
    return ranges

min(start for start, length in range_apply_maps(seed_ranges, maps))

137516820